In [ ]:
# from comet_ml import Experiment

# # Create an experiment with your api key
# experiment = Experiment(
#     api_key="4uUkmlOzv4WCej0LtoWtr5Pzj",
#     project_name="DGCNN_project_dl_bia",
#     workspace="nachinkin31",
# )

In [ ]:
# experiment.add_tag("model with noise")

In [ ]:
import os
import sys
import glob
import numpy as np
import pickle
from tqdm import tqdm

import pandas as pd

from sklearn.metrics import precision_score, roc_auc_score

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys

sys.path.append('../pytorch')

In [ ]:
!nvidia-smi

In [ ]:
import imp
import torch

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

import utils.metrics as metrics
imp.reload(metrics)
from utils.metrics import *

functions for loading and transforming data into point clouds

In [ ]:
def binary_3dmaps_to_point_cloud_and_labels(brain, mask, size = 256):
    """ 
    Transforms 3d tensors of brain and hippocampus into pointcloud and labels for it. Both only coordinates and 
    coordinates + intensity modes are suppoted
      Args:
          image_brain: torch tensor of size [size,size,size] with 1 at the positions with brain and 0 otherwise
          image_hypo: torch tensor of size [size,size,size] with 1 at the positions with hippocampus and 0 otherwise
          size: size of the input tensors along each direction, default = 256
          seg: torch tensor of size [size,size,size] with intensities of brain, default None
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    if type(size) == int:
        grid_x, grid_y, grid_z = torch.meshgrid(torch.tensor(range(size)),\
                                                torch.tensor(range(size)),\
                                                torch.tensor(range(size)))
    else:
        grid_x, grid_y, grid_z = torch.meshgrid(torch.tensor(range(size[0])),\
                                                torch.tensor(range(size[1])),\
                                                torch.tensor(range(size[2])))
        
    new = torch.cat((grid_x.unsqueeze(-1).float(), 
                     grid_y.unsqueeze(-1).float(),
                     grid_z.unsqueeze(-1).float(), 
                     torch.tensor(brain).float().unsqueeze(-1).float()), -1)
    pc_fcd = new[mask==1,:]
    fcd_len = pc_fcd.shape[0]
    idx = np.random.choice(range(fcd_len),fcd_len//100,replace = False)
    pc_fcd = pc_fcd[idx]
    
    
    pc_brain_without_fcd = new[(mask==0)*(brain != 0),:]
    no_fcd_len = pc_brain_without_fcd.shape[0]
    idx = np.random.choice(range(no_fcd_len),no_fcd_len//100,replace = False)
    pc_brain_without_fcd = pc_brain_without_fcd[idx]
    return torch.cat([pc_fcd,pc_brain_without_fcd]),\
np.array([1] * pc_fcd.shape[0] + [0] * pc_brain_without_fcd.shape[0])

In [ ]:
def fcd_filename_to_pc_and_labels(file, file_mask, size = 256):
    """ 
    Procceses filename of brain and mask into pointcloud with labels
      Args:
          file: path to brain file
          file_mask: path to mask file
          size: size of the input tensors along each direction is 256, but it can be maxpulled to size. Default = 256
          segfile: file with segmentation path
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    
    brain = load_nii_to_array(file)
      
    mask = load_nii_to_array(file_mask)
    
    pc,labels = binary_3dmaps_to_point_cloud_and_labels(brain, mask, size = size)
    
    return pc, labels

# Example of creation of Data for experiment

In [ ]:
SIZE = (316, 374, 309)
UPSAMPLE_RATE = 10
POSTFIX = '_exp1'

In [ ]:
pcs, labels = [], []
means_and_stds = []
for file in tqdm(glob.glob('../pytorch/croped_new_dataset/fcd_brains/fcd_*1.nii.gz')):
    peace = file.split('/')[-1]
    try:
        file_mask = glob.glob(f'../pytorch/croped_new_dataset/masks/mask_{peace}*')[0]
    except Exception:
        pass
    pc, label = fcd_filename_to_pc_and_labels(file, file_mask,
                                  size = SIZE)
    pc = np.array(pc.detach(),dtype = float)
    means_and_stds.append([pc.mean(0), pc.std(0)])
    pcs.append(pc)
    labels.append(label)

In [ ]:
sc_labels = [np.array(0)]*len(labels)

In [ ]:
for e in tqdm(range(len(pcs))):
    TEST_DATA_INDEXES = [e]
    std = np.array([x[1] for x in (means_and_stds[:e] + means_and_stds[e+1:])]).mean(0)
    mean = np.array([x[0] for x in (means_and_stds[:e] + means_and_stds[e+1:])]).mean(0)
    pcs_test,labels_test, sc_labels_test = ([(pcs[i]-mean)/std for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,
                                            [labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,
                                            [sc_labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE)
    pcs_train,labels_train, sc_labels_train = ([(pcs[i]-mean)/std for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\
                                            [labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\
                                            [sc_labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE)
    
    data_test = (pcs_test,labels_test, sc_labels_test)
    data_train = (pcs_train,labels_train, sc_labels_train)
    with open(f'../pytorch/data/BrainData/test_data{POSTFIX}_{e}.pkl', 'wb') as f:
        pickle.dump(data_test, f)
    with open(f'../pytorch/data/BrainData/trainval_data{POSTFIX}_{e}.pkl', 'wb') as f:
        pickle.dump(data_train, f)

In [ ]:
train_data = PCDataset(pcs_train,labels_train, num_point_cloud=10000)
test_data = PCDataset(pcs_test,labels_test, num_point_cloud=10000)

# Training

In [ ]:
from DGCNN import DGCNN_semseg
from train_eval import train_val, val_step

In [ ]:
DEVICE = 0

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(DEVICE)
torch.cuda.current_device()

In [ ]:
num_point_cloud = 20000
n_epochs = 12
batch_size = 1

In [ ]:
# if you use the pretrained model then uncommemt

# path_to_model = "models/dgcnn_new_dataset_noise.pth"

In [ ]:
#add argument std to train_val and uncomment if you want to train the models with the noise 

#std = 0.005

In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [ ]:
POSTFIX = '_exp1'

In [ ]:
#metrics to save
result_dict = {"BCE": [], "IoU": [], "ROC-AUC": [], "Dice": []}

In [ ]:
dir_model_to_save = "models/probnik"

In [ ]:
for e in tqdm(range(len(pcs))):
    print("-------------------------------------")
    print(f"training on {POSTFIX}_{e}:")
    with open(f'../pytorch/data/BrainData/test_data{POSTFIX}_{e}.pkl', 'rb') as f:
        pcs_test,labels_test,_ = pickle.load(f)
    with open(f'../pytorch/data/BrainData/trainval_data{POSTFIX}_{e}.pkl', 'rb') as f:
        pcs_train,labels_train,_ = pickle.load(f)
    #creating dataset
    train_data = PCDataset(pcs_train,labels_train, num_point_cloud)
    test_data = PCDataset(pcs_test,labels_test, num_point_cloud)
    
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
    
    pos_frac = np.sum([np.sum(el == 1) for el in labels_train]) / np.sum([len(el) for el in labels_train])
    pos_weight = torch.FloatTensor([(1 - pos_frac)/pos_frac]).to(device)
    
    model = DGCNN_semseg(k=20, emd_dims=512, dropout=0.5)
    if path_to_model is not None:
        model.load_state_dict(torch.load(path_to_model, map_location='cpu'))
    model.to(device)
    
    opt = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    metrics_dict = train_val(model, opt, criterion, trainloader, valloader,
                         n_epochs=n_epochs, is_train=True, val_step_func=val_step)
    
    result_dict["BCE_noise"].append(metrics_dict["loss"])
    result_dict["IoU_noise"].append(metrics_dict["iou"])
    result_dict["ROC-AUC_noise"].append(metrics_dict["roc"])
    result_dict["Dice_noise"].append(metrics_dict["dice"])
    
    #path_to_save
    path=os.path.join(dir_model_to_save, f"dgcnn_{POSTFIX}_{e}.pth")
    torch.save(model.state_dict(), path)
    del model
    del train_data, test_data
    

In [ ]:
df_pretrained = pd.DataFrame(result_dict, index=[f"{POSTFIX}_{e}" for e in range(len(pcs))])

df_pretrained.to_csv("metrics/exmaple.csv")